In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Niranjan kumar\\mldemo\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Niranjan kumar\\mldemo'

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path : Path
    test_data_path: Path
    model_path: Path
    all_params:dict
    metric_file_name: Path
    target_column : dict

In [40]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json


In [41]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.Logistic_Regression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path = config.data_path,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema,
       
        
        )

        return model_evaluation_config

In [42]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
from urllib.parse import urlparse
import numpy as np
import joblib

In [80]:
class ModelEvaluation:
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual, pred):
        accuracy = accuracy_score(actual,pred)
        clas_report = classification_report(actual,pred)
        conf_matrix= confusion_matrix(actual,pred)
        return accuracy, clas_report, conf_matrix
    
    def save_results(self):
        data = pd.read_csv(self.config.data_path)
        test_data = pd.read_csv(self.config.test_data_path, usecols=lambda column: column != 'id')
        model = joblib.load(self.config.model_path)
        
        target_column_schema =list(self.config.target_column.keys())[0]
    
        label_encoder = LabelEncoder()
        data[target_column_schema] = label_encoder.fit_transform(data[target_column_schema])
        test_data[target_column_schema] = label_encoder.transform(test_data[target_column_schema])

        data[target_column_schema]
    
        test_x = test_data.drop(columns=[target_column_schema],axis =1)

        test_y = test_data[target_column_schema]
    
        predicted_qualities = model.predict(test_x)
        
        (accuracy, clas_report, conf_matrix) = self.eval_metrics(test_y, predicted_qualities)
        conf_matrix = conf_matrix.tolist()
        
    # Saving ,metrics as local
        scores = {'Accuracy': accuracy, 'clas_report': clas_report, 'conf_matrix': conf_matrix}
        save_json(path= Path(self.config.metric_file_name),data = scores)
        

In [81]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
    
    
except Exception as e:
    raise e

[2024-03-11 00:38:44,010: INFO: common: yaml file: C:\Users\Niranjan kumar\mldemo\config\config.yaml loaded successfully]
[2024-03-11 00:38:44,013: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-11 00:38:44,020: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-11 00:38:44,023: INFO: common: created directory at: artifacts]
[2024-03-11 00:38:44,025: INFO: common: created directory at: artifacts/model_evaluation]
     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
0          12.21         14.09           78.78     462.00             0.08   
1          13.66         15.15           88.27     580.60             0.08   
2          12.83         15.73           82.89     506.90             0.09   
3          11.06         17.12           71.25     366.50             0.12   
4          19.19         15.94          126.30    1157.00             0.09   
..           ...           ...             ...        ...              ...   
1